In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
coef = np.array([-8.5,7.05,-0.02])  # This is the unnormalized coeffeicients of theta/2+0.2 for 100*CTR 

In [10]:
theta_star = (-coef[1]/(2*coef[0])-0.2)*2
tmp_star = theta_star/2+0.2
mu_star = (coef[0]*tmp_star**2+coef[1]*tmp_star+coef[2])/100
v_star = mu_star*(1-mu_star)

def treat_outcome(theta,M): #average outcome of M samples at theta
    tmp = theta/2+0.2
    prob = (coef[0]*tmp**2+coef[1]*tmp+coef[2])/100
    res = np.sum(np.random.binomial(size=M,n=1,p=prob))/M
    return res

In [11]:
grad_est_right = abs((treat_outcome(1,100000)-treat_outcome(0.95,100000))*20)
grad_est_left = abs((treat_outcome(0,100000)-treat_outcome(0.05,100000))*20)
grad_est_max = max(grad_est_right,grad_est_left)
rho = -np.log(grad_est_max)/np.log(1.02e9)
print(grad_est_max,rho) 
#estimate rho, if needed

0.027399999999999994 0.17341755737773681


In [42]:
def zeroth_order_opt(total_iter,M,rho,stepsize_cons): #M = m/10 in paper.
    sum_of_outcome = 0
    theta = 0.5
    theta_list = []
    value_est = np.zeros((total_iter))
    for t in range(total_iter):
        c = (t+1)**(-1/5)
        c = min(min(c,theta),1-theta)/3
        theta_plus_1 = theta+c
        theta_minus_1 = theta-c
        theta_plus_2 = theta+3*c
        theta_minus_2 = theta-3*c
        mu_tmp_1 = treat_outcome(theta_plus_1,9*M)
        mu_tmp_0 = treat_outcome(theta_minus_1,9*M)
        mu_tmp_3 = treat_outcome(theta_plus_2,M)
        mu_tmp_2 = treat_outcome(theta_minus_2,M)
        #switch between central FD method and Algorithm 1

        #svalue_est[t] = (mu_tmp_1+mu_tmp_0)/2
        value_est[t] = 9*(mu_tmp_1+mu_tmp_0)/16-(mu_tmp_3+mu_tmp_2)/16 
        sum_of_outcome += value_est[t]
        grad_est = (-mu_tmp_3+mu_tmp_2+27*mu_tmp_1-27*mu_tmp_0)/(48*c)
        #grad_est = (mu_tmp_1-mu_tmp_0)/(2*c)
        theta = theta+grad_est*stepsize_cons/((t+1)**(1-rho))
        theta = min(max(theta,0.05),0.95)
        theta_list.append(theta)
        
    return sum_of_outcome,theta,value_est,theta_list

In [43]:
T = 100000
M = 20
tot_sample = T*M*20 #20 should be 2 if you are using FDC

In [ ]:
clt_iter = 1000  #replications
clt_res = np.zeros((clt_iter))
theta_ave = 0
mu_ave = 0
for i in range(clt_iter):
    sum_of_outcome,theta_hat,value_est,theta_list = zeroth_order_opt(T,M,0,30)
    theta_list = np.array(theta_list)
    for t in range(int(0.1*T)):
        theta_ave += theta_list[T-1-t]
    theta_ave = theta_ave/(int(0.1*T))
    tmp = theta_ave/2+0.2
    mu_ave = (coef[0]*tmp**2+coef[1]*tmp+coef[2])/100
    v_ave = mu_ave*(1-mu_ave)
    clt_res[i] = np.sqrt(tot_sample)*(sum_of_outcome/T-mu_ave)/(5/4*np.sqrt(v_ave))

In [23]:
from scipy.optimize import curve_fit
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

#remember to change the algorithm if you want results of FDC

# plt.figure(dpi = 100)
# bin_heights, bin_borders, _ = plt.hist(clt_res,range=(-7,3),bins=50, label='histogram',edgecolor='r',alpha=0.5)
# bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
# popt, _ = curve_fit(gaussian, bin_centers, bin_heights, p0=[1., 0., 1.])

# x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000)
# plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fitted Gaussian curve')
# plt.xlabel('Normalized value')
# plt.ylabel('Frequency')
# plt.title('Normalized results of FDC with 1000 replications')
# plt.legend()
# plt.savefig('fdc')

In [ ]:
from scipy.optimize import curve_fit
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

plt.figure(dpi = 100)
bin_heights, bin_borders, _ = plt.hist(clt_res,range=(-5,5),bins=50, label='histogram',edgecolor='r',alpha=0.5)
bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
popt, _ = curve_fit(gaussian, bin_centers, bin_heights, p0=[1., 0., 1.])

x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000)
plt.plot(x_interval_for_fit, gaussian(x_interval_for_fit, *popt), label='fitted Gaussian curve')
plt.xlabel('Normalized value')
plt.ylabel('Frequency')
plt.title('Normalized results of Algorithm 1 with 1000 replications')
plt.legend()
plt.savefig('za')